In [4]:
import cv2
import numpy as np

# Cargar la imagen
ruta_imagen = "/home/sebastian/Documentos/analisisNETCDF/datosGraficados/componentes_conectados/componentes conectados/2021-01-01.png"
imagen = cv2.imread(ruta_imagen)

# Convertir la imagen a escala de grises
imgGris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

# Aplicar un filtro de color para detectar las nubes blancas
mascara_color = cv2.inRange(imagen, (200, 200, 200), (255, 255, 255))

# Aplicar umbralización para convertir la imagen en una imagen binaria
_, mascara = cv2.threshold(mascara_color, 200, 255, cv2.THRESH_BINARY)

# Etiquetar los componentes conectados en la imagen binaria
componentes_etiquetados = cv2.connectedComponents(mascara)[1]

# Obtener los contornos de los componentes conectados
contornos, _ = cv2.findContours((componentes_etiquetados > 0).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Obtener el centro de la imagen
alto, ancho = imgGris.shape
centro_x = ancho // 2
centro_y = alto // 2

# Definir los cuadros a ignorar
cuadros_ignorar = [
    (x1, y1, x2, y2),  # Especifica las coordenadas (x1, y1) y (x2, y2) de cada cuadro a ignorar
    # Agrega más cuadros si es necesario
]

# Función para verificar si un punto está dentro de algún cuadro
def punto_en_cuadro(x, y):
    for cuadro in cuadros_ignorar:
        x1, y1, x2, y2 = cuadro
        if x1 <= x <= x2 and y1 <= y <= y2:
            return True
    return False

# Calcular la distancia de cada contorno al centro de la imagen y verificar si está dentro de algún cuadro
def distancia_al_centro(contorno):
    M = cv2.moments(contorno)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    if punto_en_cuadro(cX, cY):
        return np.inf  # Ignorar contornos dentro de los cuadros
    return np.sqrt((cX - centro_x)**2 + (cY - centro_y)**2)

# Ordenar los contornos según su distancia al centro
contornos_ordenados = sorted(contornos, key=distancia_al_centro, reverse=True)

# Crear una copia de la imagen original para dibujar los contornos
imagen_contornos = imagen.copy()

# Dibujar contornos en forma de anillos verdes alrededor de los componentes conectados
for contorno in contornos_ordenados:
    area = cv2.contourArea(contorno)
    if area > 1000:  # Ajusta este valor según el tamaño de las nubes que deseas detectar
        cv2.drawContours(imagen_contornos, [contorno], -1, (0, 255, 0), 2)

# Guardar la imagen con los contornos dibujados
ruta_guardado = "/home/sebastian/Documentos/analisisNETCDF/objetos/componentes_conectados_contornos_ordenados.png"
if cv2.imwrite(ruta_guardado, imagen_contornos):
    print("Imagen con contornos de componentes conectados guardada exitosamente en:", ruta_guardado)
else:
    print("No se pudo guardar la imagen.")

NameError: name 'x1' is not defined